# ClusterFlix

ClusterFlix is my version of Netflix's recommender system. It uses an NLP (Natural Language Processing) model and a K-Means Clustering model to group Netflix movies and TV shows by their plot description, genre, lead actor or actress, director, and the country the movie or TV show was filmed in. Users can utilize my recommender model to find movies or TV shows that are similar to their favourite films.

#  
## Packages

In [1]:
import pandas as pd
import numpy as np
import spacy
import sklearn

#  
## Data Set

There is a total of 6234 movies / TV shows in this data set collected from [Kaggle](https://www.kaggle.com/shivamb/netflix-shows). 

Each row contains the following information: **type** (Movie or TV Show), **title**, **director**, **cast**, **country**, **rating** (ex. PG, PG-13, R, etc.), **listed_in** (genre), and plot **description**.

In [2]:
# Load in the data from csv file
netflix = pd.read_csv('netflix.csv').drop(['show_id','date_added','release_year','duration'], axis=1)
netflix_df = netflix.copy()
netflix_df.head()

,type,title,director,cast,country,rating,listed_in,description
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,TV-MA,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,TV-Y7-FV,Kids' TV,"With the help of three human allies, the Autob..."
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,TV-Y7,Kids' TV,When a prison ship crash unleashes hundreds of...
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,TV-14,Comedies,When nerdy high schooler Dani finally attracts...


#   
## Natural Language Processing (NLP) Model

For the NLP portion of this project, I will first convert all plot descriptions to word vectors so they can be processed by the NLP model. Then, the similarity between all word vectors will be calculated using cosine similarity (measures the angle between two vectors, resulting in a score between -1 and 1, corresponding to complete oppositses or perfectly similar vectors). Finally, I will extract the 5 movies or TV shows with the most similar plot description to a given movie or TV show.

In [3]:
# Load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

In [4]:
# Create word vectors for all movie and TV show descriptions
with nlp.disable_pipes():
    vectors = np.array([nlp(film.description).vector for idx, film in netflix_df.iterrows()])

In [5]:
# Function to analyze how similar two word vectors are
def cosine_similarity(a, b):
    return np.dot(a, b)/np.sqrt(a.dot(a)*b.dot(b))

In [6]:
# Calculate the mean for all word vectors
vec_mean = vectors.mean(axis=0)

# Subtract the mean from the vectors
centered = vectors - vec_mean 

In [7]:
# Function to get the indices of the five most similar descriptions
def get_similar_description_indices(description_vec):
    
    # Calculate similarities between given description and other descriptions in the dataset
    sims = np.array([cosine_similarity(description_vec - vec_mean, vec) for vec in centered])
    
    # Get the indices of the five most similar descriptions
    most_similar_index = np.argsort(sims)[-6:-1]
    
    return most_similar_index

In [8]:
# Create array of lists containing indices of five most similar descriptions
similar_indices = np.array([get_similar_description_indices(vec) for vec in vectors])

 
### Test NLP Model

To test my NLP model, I will look at the most similar plot descriptions to one of my favourite movies, *Catch Me If You Can*, directed by Steven Spielberg and starring Leonardo DiCaprio and Tom Hanks.

In [9]:
test_index = netflix.index[netflix.title == "Catch Me If You Can"][0]

print("Chosen Movie/TV Show")
print(netflix_df.title[test_index] + ': ' + netflix_df.description[test_index] + '\n')
print("Top Recommendations")
print(netflix_df.title[similar_indices[test_index][4]] + ': ' + netflix_df.description[similar_indices[test_index][4]] + '\n')
print(netflix_df.title[similar_indices[test_index][3]] + ': ' + netflix_df.description[similar_indices[test_index][3]] + '\n')
print(netflix_df.title[similar_indices[test_index][2]] + ': ' + netflix_df.description[similar_indices[test_index][2]] + '\n')
print(netflix_df.title[similar_indices[test_index][1]] + ': ' + netflix_df.description[similar_indices[test_index][1]] + '\n')
print(netflix_df.title[similar_indices[test_index][0]] + ': ' + netflix_df.description[similar_indices[test_index][0]] + '\n')

Chosen Movie/TV Show
Catch Me If You Can: An FBI agent makes it his mission to put cunning con man Frank Abagnale Jr. behind bars. But Frank not only eludes capture, he revels in the pursuit.

Top Recommendations
MI-5: When a terrorist escapes custody, MI-5 director Harry Pearce becomes the scapegoat. But the wily veteran still has some spycraft tricks up his sleeve.

The Blacklist: After turning himself in, a brilliant fugitive offers to help the FBI bag other baddies, but only if rookie profiler Elizabeth Keen is his partner.

The Bounty Hunter: Bounty hunter Milo Boyd finds his latest assignment downright satisfying as he learns that the bail-jumper he must chase down is his ex-wife, Nicole.

War: When his partner is killed and all clues point to a notorious assassin, FBI agent Jack Crawford sets out for revenge in this taut action-thriller.

Donnie Brasco: When FBI agent Joe Pistone goes undercover to infiltrate the mob, he starts to identify with the made men he's charged with tak

As you can see, my NLP model detected a lot of similar key words like 'FBI' from the descriptions. The descriptions also appear to be semantically related as they all appear to be related to crime.

#   
#   
## K-Means Clustering Model

Prior to creating the k-means clustering model, I will perform the following data cleaning and feature engineering tasks:
- Fill missing values with most common entry for the column (ex. fill missing country entries as 'United States')
- Convert columns with multiple values in a cell to a list and only take the first value (ex. take only the lead actor or actress from each cast list)
- Encode all categorical variables

### Data Cleaning 

In [10]:
# Fill missing country entries with United States
netflix_df.fillna('Other', inplace=True)

# Change director, cast, country, and listed_in columns from type str to lists
netflix_df.director = netflix_df.director.str.split(', ').tolist()
netflix_df.cast = netflix_df.cast.str.split(', ').tolist()
netflix_df.country = netflix_df.country.str.split(', ').tolist()
netflix_df.listed_in = netflix_df.listed_in.str.split(', ').tolist()

#   
### Feature Reduction

In [11]:
# Array of all unique directors, cast members, countries, ratings, and genres
all_directors = netflix_df['director'].explode().unique()
all_cast = netflix_df['cast'].explode().unique()
all_countries = netflix_df['country'].explode().unique()
all_ratings = netflix_df['rating'].unique()
all_genres = netflix_df['listed_in'].explode().unique()

len(all_directors), len(all_cast), len(all_countries), len(all_ratings),len(all_genres)

(3656, 27406, 107, 15, 42)

There are 3656 directors, 27406 actors / actresses, and 107 countries  in the data set which are too many features to include in a K-Means clustering model. Thus, I will reduce the number of features by only taking the primary director, lead actor/actress, and primary country for each movie or TV show. Then, I will count encode each of these features by replacing each categorical value with the number of times it appears in the dataset.

I will use one-hot encoding encoding to encode ratings and genres (listed_in) since there are only 15 ratings and 42 genres. One-hot-encoding creates new columns indicating the presence (1) or absence (0) of each possible value in the data. Since a movie or TV show can belong to more than one genre, I will use a Multi Label Binarizer for rating.

In [12]:
# Retain primary director, lead actor/actress, and primary country
for inx in range(len(netflix_df)):
    netflix_df['director'][inx] = netflix_df['director'][inx][0]
    netflix_df['cast'][inx] = netflix_df['cast'][inx][0]
    netflix_df['country'][inx] = netflix_df['country'][inx][0]
    inx += 1

In [13]:
feature_reduced_df = netflix_df.copy()
feature_reduced_df.head()

,type,title,director,cast,country,rating,listed_in,description
0,Movie,Norm of the North: King Sized Adventure,Richard Finn,Alan Marriott,United States,TV-PG,"[Children & Family Movies, Comedies]",Before planning an awesome wedding for his gra...
1,Movie,Jandino: Whatever it Takes,Other,Jandino Asporaat,United Kingdom,TV-MA,[Stand-Up Comedy],Jandino Asporaat riffs on the challenges of ra...
2,TV Show,Transformers Prime,Other,Peter Cullen,United States,TV-Y7-FV,[Kids' TV],"With the help of three human allies, the Autob..."
3,TV Show,Transformers: Robots in Disguise,Other,Will Friedle,United States,TV-Y7,[Kids' TV],When a prison ship crash unleashes hundreds of...
4,Movie,#realityhigh,Fernando Lebrija,Nesta Cooper,United States,TV-14,[Comedies],When nerdy high schooler Dani finally attracts...


#  
### Feature Engineering (Categorical Encoding)

* Use the MultiLabelBinarizer to encode the genres the movies or TV shows are listed in (each entry can belong to multiple genres)
* One-hot encode rating
* Count encode the primary director, lead actor or actress, and country of each movie / TV show

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

# Create the MultiLabelBinarizer 
mlb = MultiLabelBinarizer()

# Encode each genre and join to dataframe
mlb_df = feature_reduced_df.join(pd.DataFrame(mlb.fit_transform(feature_reduced_df.pop('listed_in')),
                                              columns=mlb.classes_,
                                              index=feature_reduced_df.index))

In [15]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to rating column
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_rating = pd.DataFrame(OH_encoder.fit_transform(mlb_df[['rating']]), columns=all_ratings)

# One-hot encoding removed index; put it back
OH_rating.index = mlb_df.index

# Add one-hot encoded columns to data frame
mlb_df = mlb_df.join(OH_rating)

In [16]:
import category_encoders as ce

# Create the count encoder
count_enc = ce.CountEncoder()

# Count encode director, cast, and country columns
count_encoded = count_enc.fit_transform(mlb_df[['director','cast','country']])

# Rename the columns with the _count suffix, and join to dataframe
netflix_encoded_df = mlb_df.join(count_encoded.add_suffix("_count"))

In [17]:
netflix_encoded_df = netflix_encoded_df.drop(['type','title','director','cast','country','rating','description'], axis=1)
netflix_encoded_df.head()

,Action & Adventure,Anime Features,Anime Series,British TV Shows,Children & Family Movies,Classic & Cult TV,Classic Movies,Comedies,Crime TV Shows,Cult Movies,...,PG-13,TV-G,PG,G,Other,UR,NC-17,director_count,cast_count,country_count
0,0,0,0,0,1,0,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,2302
1,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1969,2,483
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1969,1,2302
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1969,1,2302
4,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,2302


#    
### Create (K-Means) Clusters 

I will create a k-means clustering model that will group the 6234 movies/TV shows into 500 clusters. After initializing the model, cluster predictions will be made and attached to the original data frame to show each movie/TV show and the cluster they belong to.

In [18]:
from sklearn.cluster import KMeans

# Create K-Means Model
modelkmeans = KMeans(n_clusters=500, init='k-means++', n_init=10).fit(netflix_encoded_df)

# Form cluster predictions using K-Means Model
predictions = modelkmeans.predict(netflix_encoded_df)

# Convert cluster predictions to data frame
predictions_df = pd.DataFrame(predictions, columns=['cluster'])

# Attach cluster predictions to original data frame
netflix.insert(len(netflix.columns), column = 'cluster', value = predictions_df.cluster)

In [19]:
netflix.head()

,type,title,director,cast,country,rating,listed_in,description,cluster
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,445
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,TV-MA,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,489
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,TV-Y7-FV,Kids' TV,"With the help of three human allies, the Autob...",136
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,TV-Y7,Kids' TV,When a prison ship crash unleashes hundreds of...,303
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,TV-14,Comedies,When nerdy high schooler Dani finally attracts...,370


#    
### Test K-Means Clustering Model 

I will test the k-means clustering model using one of my favourite TV shows, *Breaking Bad*.

In [26]:
# Get cluster number from given movie or TV show
cluster_num = netflix[netflix.title=='Breaking Bad'].cluster.item()

# View cluster the movie or TV show belongs to
netflix[netflix.cluster == cluster_num]

,type,title,director,cast,country,rating,listed_in,description,cluster
278,TV Show,Unbelievable,NaN,"Toni Collette, Merritt Wever, Kaitlyn Dever, D...",United States,TV-MA,"Crime TV Shows, TV Dramas",After a young woman is accused of lying about ...,3
1459,TV Show,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",Organized crime enforcer Miles Daly strives to...,3
3817,TV Show,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",Ride along for a dramatized version of the rea...,3
5741,TV Show,House of Cards,NaN,"Kevin Spacey, Robin Wright, Kate Mara, Corey S...",United States,TV-MA,"TV Dramas, TV Thrillers",A ruthless politician will stop at nothing to ...,3
5827,TV Show,Hap and Leonard,NaN,"James Purefoy, Michael Kenneth Williams, Jimmi...",United States,TV-MA,"Crime TV Shows, TV Dramas","In late-1980s Texas, two best friends get invo...",3
5850,TV Show,Better Call Saul,NaN,"Bob Odenkirk, Jonathan Banks, Michael McKean, ...",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas","This Emmy-nominated prequel to ""Breaking Bad"" ...",3
5905,TV Show,Aquarius,NaN,"David Duchovny, Gethin Anthony, Grey Damon, Em...",United States,TV-MA,"Crime TV Shows, TV Dramas","Amid the turmoil of 1960s LA, two cops and a p...",3
5982,TV Show,Twin Peaks,NaN,"Kyle MacLachlan, Michael Ontkean, Mädchen Amic...",United States,TV-MA,"Classic & Cult TV, Crime TV Shows, TV Dramas","""Who killed Laura Palmer?"" is the question on ...",3
5997,TV Show,Marvel's The Punisher,NaN,"Jon Bernthal, Ebon Moss-Bachrach, Ben Barnes, ...",United States,TV-MA,"Crime TV Shows, TV Action & Adventure, TV Thri...",A former Marine out to punish the criminals re...,3
6023,TV Show,Dexter,NaN,"Michael C. Hall, Jennifer Carpenter, David Zay...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries","By day, mild-mannered Dexter is a blood-spatte...",3


As you can see, this cluster primarily contains American Crime TV Shows and TV Dramas.

#   
## Future Directions

I plan to create a Flask app that will allow users to input a movie or TV show and will provide recommendations based on the plot description, director, lead actor/actress, genre, and country it was produced in.